To use this code, you must have Emap2sec+ predictions in a numpy file (.npy). 

In [ ]:
%matplotlib inline
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import mrcfile

Use this to check where predictions are placed.

In [ ]:
orig = np.load('9515_E_prediction.npy')
map_0 = np.zeros((orig.shape[0], orig.shape[1], orig.shape[2]), dtype=float)
for i in range(orig.shape[0]):
    for j in range(orig.shape[1]):
        for k in range(orig.shape[2]):
            map_0[i, j, k] = orig[i, j, k, 1]
            if (map_0[i, j, k] > 0):
                print(i, j, k)

In [3]:
data = np.load('9515_E_prediction.npy')
np.set_printoptions(precision=3)

print(data.shape[0])
print(data.shape[1])
print(data.shape[2])
print(data.shape[3])
print(np.count_nonzero(data))

98
80
125
4
87248


At its current state, we have to change each index manually after checking if the predictions are placed at even or odd indices.

In [9]:
%%time
#Single odd/even index
for i in range(0, data.shape[0]-1, 2):
    for j in range(1, data.shape[1]-1, 2):
        for k in range(1, data.shape[2]-1, 2):
            for l in range(data.shape[3]):
                if (data[i, j, k, l] == 0):
                    avg = 0
                    avg += data[i+1, j, k, l]
                    avg += data[i-1, j, k, l]
                    avg = avg / float(2)
                    data[i, j, k, l] = avg

                    
for j in range(0, data.shape[1]-1, 2):
    for i in range(1, data.shape[0]-1, 2):
        for k in range(1, data.shape[2]-1, 2):
            for l in range(data.shape[3]):
                if (data[i, j, k, l] == 0):
                    avg = 0
                    avg += data[i, j+1, k, l]
                    avg += data[i, j-1, k, l]
                    avg = avg / float(2)
                    data[i, j, k, l] = avg


for k in range(0, data.shape[2]-1, 2):
    for i in range(1, data.shape[0]-1, 2):
        for j in range(1, data.shape[1]-1, 2):
            for l in range(data.shape[3]):
                if (data[i, j, k, l] == 0):
                    avg = 0
                    avg += data[i, j, k+1, l]
                    avg += data[i, j, k-1, l]
                    avg = avg / float(2)
                    data[i, j, k, l] = avg


#Two odd/even indices
for i in range(0, data.shape[0]-1, 2):
    for j in range(0, data.shape[1]-1, 2):
        for k in range(1, data.shape[2]-1, 2):
            for l in range(data.shape[3]):
                if (data[i, j, k, l] == 0):
                    avg = 0
                    avg += data[i+1, j, k, l]
                    avg += data[i-1, j, k, l]
                    avg += data[i, j+1, k, l]
                    avg += data[i, j-1, k, l]
                    avg = avg / float(4)
                    data[i, j, k, l] = avg


for i in range(0, data.shape[0]-1, 2):
    for k in range(0, data.shape[2]-1, 2):
        for j in range(1, data.shape[1]-1, 2):
            for l in range(data.shape[3]):
                if (data[i, j, k, l] == 0):
                    avg = 0
                    avg += data[i+1, j, k, l]
                    avg += data[i-1, j, k, l]
                    avg += data[i, j, k+1, l]
                    avg += data[i, j, k-1, l]
                    avg = avg / float(4)
                    data[i, j, k, l] = avg


for j in range(0, data.shape[1]-1, 2):
    for k in range(0, data.shape[2]-1, 2):
        for i in range(1, data.shape[0]-1, 2):
            for l in range(data.shape[3]):
                if (data[i, j, k, l] == 0):
                    avg = 0
                    avg += data[i, j+1, k, l]
                    avg += data[i, j-1, k, l]
                    avg += data[i, j, k+1, l]
                    avg += data[i, j, k-1, l]
                    avg = avg / float(4)
                    data[i, j, k, l] = avg


#Three odd/even indices
for i in range(0, data.shape[0]-1, 2):
    for j in range(0, data.shape[1]-1, 2):
        for k in range(0, data.shape[2]-1, 2):
            for l in range(data.shape[3]):
                if (data[i, j, k, l] == 0):
                    avg = 0
                    avg += data[i+1, j, k, l]
                    avg += data[i-1, j, k, l]
                    avg += data[i, j+1, k, l]
                    avg += data[i, j-1, k, l]
                    avg += data[i, j, k+1, l]
                    avg += data[i, j, k-1, l]
                    avg = avg / float(6)
                    data[i, j, k, l] = avg

CPU times: user 7min 1s, sys: 668 ms, total: 7min 2s
Wall time: 7min 3s


In [14]:
np.save('interpolated_prediction_9515_E', data)

In [15]:
interpolated = np.load('interpolated_prediction_9515_E.npy')

In [ ]:
# to ensure maps were correctly saved
for i in range(interpolated.shape[0]):
    for j in range(interpolated.shape[1]):
        for k in range(interpolated.shape[2]):
            if (orig[i, j, k, 0] == 0):
                if (chain_a[i, j, k, 0] > 0):
                    print(i, j, k, chain_a[i, j, k, ])

To visualize, run the code for each map.

In [7]:
def conv_prediction_to_mrc(arr,reffile,keyidx,outmapfile):      
    with mrcfile.open(reffile) as mrc:
        nx,ny,nz = arr.shape[0], arr.shape[1], arr.shape[2]
        mx,my,mz,cella = mrc.header.mx,mrc.header.my,mrc.header.mz, mrc.header.cella
        
        
        mrc_new = mrcfile.new(outmapfile,overwrite=True)
        
        mrc_new.set_data(np.zeros((nz, ny, nx), dtype=np.float32))
        mrc_new.header.nxstart=arr.shape[0]
        mrc_new.header.nystart=arr.shape[1]
        mrc_new.header.nzstart=arr.shape[2]
        mrc_new.header.origin.x = mrc.header.origin.x
        mrc_new.header.origin.y = mrc.header.origin.y
        mrc_new.header.origin.z = mrc.header.origin.z
        mrc_new.header.cella['x'] = cella['x']
        mrc_new.header.cella['y'] = cella['y']
        mrc_new.header.cella['z'] = cella['z']
        
        print(nx, ny, nz)
        for i in range(nx - 2):
            for j in range(ny - 2):
                for k in range(nz - 2):
                    mrc_new.data[k, j, i] = arr[i, j, k, keyidx]
        
        vsize=mrc_new.voxel_size
        vsize.flags.writeable = True
        mrc_new.voxel_size=vsize         
        
        mrc_new.update_header_stats()
        
        print("original", mrc.voxel_size)
        mrc.print_header()

        print()
        
        print("new", mrc_new.voxel_size)
        mrc_new.print_header()

        print()
        
        mrc_new.close()

In [16]:
%%time
conv_prediction_to_mrc(interpolated, '9515_E.mrc', 3, 'interpolated_prediction_9515_E_3.mrc')

98 80 125
original (1., 1., 1.)
nx              : 125
ny              : 80
nz              : 98
mode            : 2
nxstart         : 0
nystart         : 0
nzstart         : 0
mx              : 125
my              : 80
mz              : 98
cella           : (125., 80., 98.)
cellb           : (90., 90., 90.)
mapc            : 1
mapr            : 2
maps            : 3
dmin            : 0.0
dmax            : 0.5443896651268005
dmean           : 0.008815660141408443
ispg            : 1
nsymbt          : 0
extra1          : b'\x00\x00\x00\x00\x00\x00\x00\x00'
exttyp          : b''
nversion        : 20140
extra2          : b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
origin          : (116.58, 134.889,